# Example of using PaLM API with LangChain

# Preparation

In [ ]:
pip install --user langchain==0.0.260

## Building a simple pipeline with LangChain

In [1]:
from langchain import PromptTemplate

template = """\
You are a naming consultant for new products.
Give me three examples of good names for a product.
Output only names in a comma-separated list, nothing else.

Here's an example.
product: cute pens
output: Scribble, Ink-it, Write-On

Here's the real request to you.
product: {description}
output:
"""

prompt = PromptTemplate(template=template, input_variables=['description'])

In [2]:
print(prompt.format(description='Colorful cute smartphone covers for teenagers'))

You are a naming consultant for new products.
Give me three examples of good names for a product.
Output only names in a comma-separated list, nothing else.

Here's an example.
product: cute pens
output: Scribble, Ink-it, Write-On

Here's the real request to you.
product: Colorful cute smartphone covers for teenagers
output:



In [3]:
from langchain.llms import vertexai
llm = vertexai.VertexAI(model_name='text-bison', temperature=0.4)

In [4]:
from langchain import LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [5]:
description = 'Colorful cute smartphone covers for teenagers'
print(llm_chain.run(description))

 1. Pop-Art
2. Color-Splash
3. i-Candy


In [6]:
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

In [7]:
output_parser.parse(llm_chain.run(description))

['Pop-Art', 'Bling-It', 'Glam-Case']

## Using pydantic to convert a response in a predefined object

In [8]:
from pydantic import BaseModel, Field

class ProductNames(BaseModel):
    setup: str = Field(description='product description')
    product_name1: str = Field(description='product name 1')
    product_name2: str = Field(description='product name 2')
    product_name3: str = Field(description='product name 3')

In [9]:
from langchain.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=ProductNames)

In [10]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "product description", "type": "string"}, "product_name1": {"title": "Product Name1", "description": "product name 1", "type": "string"}, "product_name2": {"title": "Product Name2", "description": "product name 2", "type": "string"}, "product_name3": {"title": "Product Name3", "description": "product name 3", "type": "string"}}, "required": ["setup", "product_name1", "product_name2", "product_name3"]}
```


In [11]:
template="""\
Answer the user query.
{format_instructions}
You are a naming consultant for new products.
Give me examples of good names for a product described as {description}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['description'],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [12]:
description = "Super tough smartphone covers that wouldn't break even if an elephant stamps on them"
output = parser.parse(llm_chain.run(description))
output


ProductNames(setup="Super tough smartphone covers that wouldn't break even if an elephant stamps on them", product_name1='Eleproof', product_name2='Tuff-Tech', product_name3='Armored Shield')

In [13]:
output.product_name1, output.product_name2, output.product_name3

('Eleproof', 'Tuff-Tech', 'Armored Shield')

## Defining a pipeline including the parser

In [14]:
from langchain.chains import TransformChain, SequentialChain

llm_chain = LLMChain(prompt=prompt, llm=llm, output_key='json_string')

def parse_output(inputs):
    text = inputs['json_string']
    return {'result': parser.parse(text)}

transform_chain = TransformChain(
    input_variables=['description'],
    output_variables=['result'],
    transform=parse_output
)

chain = SequentialChain(
    input_variables=['description'],
    output_variables=['result'],
    chains=[llm_chain, transform_chain],
)

In [15]:
description = "Super tough smartphone covers that wouldn't break even if an elephant stamps on"
chain.run(description=description)

ProductNames(setup="Super tough smartphone covers that wouldn't break even if an elephant stamps on", product_name1='Eleproof', product_name2='Tuff-Tech', product_name3='Armored Shield')